In [19]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [3]:
INP.head()

,hnum2,PPAY,hsa,MDC
0,11,1,12,23
1,11,1,12,23
2,11,1,8,8
3,11,1,11,8
4,11,7,12,4


In [27]:
INP = pd.read_csv("VTINP16_upd.csv",usecols=['hnum2','MDC','PPAY','hsa'])

In [241]:
INP[INP['MDC']==5].shape

(6343, 4)

In [253]:
RR_hosp = pd.read_excel("CaseStudy_O-D_HospMonopoly_LATTE.xlsx", sheet_name="Hosp_Destination")

In [254]:
RR_hosp.rename(columns={"RR":"RR_destination"} ,inplace=True)

In [255]:
RR_hsa = pd.read_excel("CaseStudy_O-D_HospMonopoly_LATTE.xlsx", sheet_name="HSA_Pt_Origin")

In [256]:
RR_hsa.rename(columns={"RR Collapsed Referral Region":"RR_origin"},inplace=True)

In [257]:
INP=INP.merge(RR_hosp, on='hnum2', how='left')

In [258]:
INP=INP.drop(columns=['RRName'])

In [259]:
INP=INP.merge(RR_hsa, on='hsa', how='left')

In [260]:
INP.head()

,hnum2,PPAY,hsa,MDC,HospitalName,RR_destination,HSA Name,RR_origin,Name,RR Name
0,11,1,12,23,Mount Ascutney Hospital And Health Center,RR4,White River Jct.,RR4,Randolph,RR4--Randolph
1,11,1,12,23,Mount Ascutney Hospital And Health Center,RR4,White River Jct.,RR4,Randolph,RR4--Randolph
2,11,1,8,8,Mount Ascutney Hospital And Health Center,RR4,Middlebury,RR1,Burlington,RR1--Burlington
3,11,1,11,8,Mount Ascutney Hospital And Health Center,RR4,Springfield,RR4,Randolph,RR4--Randolph
4,11,7,12,4,Mount Ascutney Hospital And Health Center,RR4,White River Jct.,RR4,Randolph,RR4--Randolph


In [261]:
INP= INP.drop(columns=['Name','RR Name'])

#### filter the dataset

In [28]:
INP["MDC"].replace(' ', np.nan, inplace=True)

In [29]:
INP.dropna(subset=['MDC'],inplace = True)

In [30]:
INP["MDC"] = INP["MDC"].astype(int)

In [31]:
INP = INP[INP['hsa']<=13]

In [32]:
PP = [1,6,7]
INP = INP.loc[INP["PPAY"].isin(PP)]

In [33]:
MDC = [5,8]
INP = INP.loc[INP["MDC"].isin(MDC)]

In [34]:
INP.head()

,hnum2,PPAY,hsa,MDC
2,11,1,8,8
3,11,1,11,8
6,11,1,12,8
9,11,1,12,8
12,11,1,12,8


In [35]:
INP["PPAY"].unique()

array([1, 6, 7])

In [36]:
RR = ['RR3','RR5','RR1','RR2','RR4']
INP = INP.loc[INP["RR_origin"].isin(RR)]

KeyError: 'RR_origin'

#### fisher test


In [37]:
Medicare_high_end = (INP["PPAY"]==1) & (INP["MDC"]==5)
Medicare_low_end = (INP["PPAY"]==1) & (INP["MDC"]==8)
Commercial_high_end = ((INP["PPAY"]==6)|(INP["PPAY"]==7)) & (INP["MDC"]==5)
Commercial_low_end = ((INP["PPAY"]==6)|(INP["PPAY"]==7)) & (INP["MDC"]==8)

In [38]:
Medicare_high_end.head()

2     False
3     False
6     False
9     False
12    False
dtype: bool

In [39]:
PM = np.array([[Medicare_high_end.sum(),Medicare_low_end.sum()],[Commercial_high_end.sum(),Commercial_low_end.sum()]])
df_f = pd.DataFrame(PM, columns=['High End Care','Low End Care'])
df_f.index = ['Medicare','Commercial']
df_f

,High End Care,Low End Care
Medicare,3831,3346
Commercial,1017,1531


In [40]:
oddscratio, pvalue =stats.fisher_exact(df_f)

In [41]:
pvalue

1.3100523346671551e-31

### OD1- high end care

In [269]:
INP_high_end_care = INP[INP['MDC']==5]

In [270]:
INP_high_end_care.shape

(5262, 8)

In [271]:
INP_high_end_care.head()

,hnum2,PPAY,hsa,MDC,HospitalName,RR_destination,HSA Name,RR_origin
20,11,1,12,5,Mount Ascutney Hospital And Health Center,RR4,White River Jct.,RR4
57,11,1,12,5,Mount Ascutney Hospital And Health Center,RR4,White River Jct.,RR4
67,11,1,12,5,Mount Ascutney Hospital And Health Center,RR4,White River Jct.,RR4
111,5,6,3,5,University of Vermont Medical Center (as of 2014),RR1,Morrisville,RR2
139,5,1,2,5,University of Vermont Medical Center (as of 2014),RR1,Burlington,RR1


In [272]:
INP_high_end = INP_high_end_care.groupby(["RR_origin","RR_destination"])['hnum2'].count().reset_index(name ='total_admission')

In [273]:
INP_high_end

,RR_origin,RR_destination,total_admission
0,RR1,RR1,2103
1,RR1,RR2,9
2,RR1,RR4,1
3,RR1,RR5,8
4,RR2,RR1,458
5,RR2,RR2,689
6,RR2,RR3,6
7,RR2,RR4,22
8,RR3,RR1,53
9,RR3,RR2,15


In [157]:
INP_high_end_care[INP_high_end_care['RR_destination']=='RR1'].shape

(2836, 8)

In [153]:
high_end_matrix = (
    INP_high_end.assign(count=1)
    .pivot_table(index="RR_origin", columns="RR_destination", values="count", aggfunc="count")
    .fillna(0)
    .astype(int)
)

print(high_end_matrix)

RR_destination  RR1  RR2  RR3  RR4  RR5
RR_origin                              
RR1               1    1    0    1    1
RR2               1    1    1    1    0
RR3               1    1    1    1    1
RR4               1    1    1    1    1
RR5               1    0    1    1    1


### low end

In [154]:
INP_low_end_care = INP[INP['MDC']==8]

In [ ]:
INP_low_end = INP_low_end_care.groupby(["RR_origin","RR_destination"])['hnum2'].count().reset_index(name ='total_admission')